# Imports


In [10]:
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install matplotlib

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 36.0 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [16]:
!pip install nbformat

  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl (18 kB)
Using cached referencing-0.36.2-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [nbformat]5/7 [jsonschema]


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import pandas as pd
import matplotlib.pyplot as plt



Matplotlib is building the font cache; this may take a moment.


In [12]:
import time
from sklearn.model_selection import train_test_split
# from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

# Loading

In [17]:
import nbformat

# Load the second uploaded notebook

# Load the second uploaded notebook
with open(notebook_path_2, 'r', encoding='utf-8') as f:
    notebook_2 = nbformat.read(f, as_version=4)

# Extract code cells from the second notebook
code_cells_2 = [cell['source'] for cell in notebook_2.cells if cell.cell_type == 'code']
code_cells_2[:3]  # Show first few code cells to understand the structure


["# Data Loader\n\nimport numpy as np\nimport pandas as pd\nimport pickle\nimport os\n\n\nclass Bunch(dict):\n    def __init__(self, **kwargs):\n        super().__init__(kwargs)\n\n        for key, value in kwargs.items():\n            setattr(self, key, value)\n\ndef _convert_to_numpy_array(dataframes: list[pd.DataFrame], target: pd.Series):\n    \n    numpy_data = []\n    for df in dataframes:\n        numpy_data.append(df.values)\n\n    return np.array(numpy_data), target.to_numpy()\n        \ndef load_tsa_data(return_X_y=False):\n\n    with open('tsa_dataset/description.txt', 'r') as fdescr:\n        descr = fdescr.read()\n\n    data_filename = os.path.join('tsa_dataset/data', 'tsa_data.pkl')\n    with open(data_filename, 'rb') as file:\n        data, target = pickle.load(file) # returns tuple[list[pd.Dataframe], pd.Series]\n\n    feature_names = data[0].columns.to_list() # list[str]\n    target_names = ['stable', 'unstable']\n\n    data, target = _convert_to_numpy_array(data, targ

In [20]:
bunch = load_tsa_data()
data, target = bunch.data, bunch.target


NameError: name 'load_tsa_data' is not defined


# Scaling Test

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time

# Scaling Test (Data Size Only)

import matplotlib.pyplot as plt

# Assuming `data` and `target` are already defined in the notebook
# If not, you need to define them based on your dataset
# Example: data, target = some_data_loading_function()

# Split data once to reuse the same test set
X_train_full, X_test, y_train_full, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

fractions = [0.1, 0.25, 0.5, 0.75, 1.0]
results = []

for frac in fractions:
    size = int(len(X_train_full) * frac)
    X_train = X_train_full[:size]
    y_train = y_train_full[:size]

    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)

    start_time = time.time()
    clf.fit(X_train, y_train)
    duration = time.time() - start_time

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    results.append((frac, size, duration, acc))
    print(f"Data Fraction: {frac:.2f}, Train Size: {size}, Time: {duration:.2f}s, Accuracy: {acc:.4f}")

# Results
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])

# Plot
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Training Size')
ax1.set_ylabel('Training Time (s)', color=color)
ax1.plot(df_results["Train Size"], df_results["Time (s)"], marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Accuracy', color=color)
ax2.plot(df_results["Train Size"], df_results["Accuracy"], marker='o', color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title("Scaling Test: MLP Training Time & Accuracy")
plt.show()

df_results


NameError: name 'data' is not defined

In [ ]:
# Display results as DataFrame
df_results = pd.DataFrame(results, columns=["Fraction", "Train Size", "Time (s)", "Accuracy"])
print(df_results)


In [ ]:
# Plot
df_results.plot(x="Train Size", y=["Time (s)", "Accuracy"], marker='o', title="Scaling Test Results")
plt.grid(True)
plt.show()
